In [1]:
import numpy as np

from synthesizer import Grid
from astropy.cosmology import Planck18 as cosmo


grid = Grid("test_grid.hdf5")

In [13]:
from synthesizer import Grid
from astropy.cosmology import Planck18 as cosmo

log10age = 6.0  # log10(age/yr)
metallicity = 0.01

# find nearest grid point
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)
line = grid.get_lines(grid_point, "He 2 1640.41A")

print(line.equivalent_width, line.luminosity, line.lam)

flux = line.get_flux(cosmo, z=5)

print(flux)

[0.15399438] Å [4.68265894e+31] erg/s [1640.41] Å
[1.72166987e-28] erg/(cm**2*s)


In [22]:
import time

import matplotlib.colors as cm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
from unyt import Hz, angstrom, arcsecond, erg, kpc, s, Msun, Myr, pc

from synthesizer import TEST_DATA_DIR
from synthesizer.emission_models import BimodalPacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.filters import FilterCollection as Filters
from synthesizer.grid import Grid
from synthesizer.kernel_functions import Kernel


from synthesizer.particle import Galaxy, Stars

gal = Galaxy(stars=Stars(ages=np.array([1e7])*Myr, metallicities=np.array([0.02]), initial_masses=[1e8]*Msun), redshift=5)


/tmp/ipykernel_3285813/2597485938.py:21: RuntimeWarning: 
Current mass of stars not provided, setting stellar_mass_weighted_age to `None`
  gal = Galaxy(stars=Stars(ages=np.array([1e7])*Myr, metallicities=np.array([0.02]), initial_masses=[1e8]*Msun), redshift=5)


In [23]:
# Get the stellar pacman model
model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_birth=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_birth=PowerLaw(slope=-0.7),
    fesc=0.1,
    fesc_ly_alpha=0.9,
    label="total",
    per_particle=True,
)

# And use it to generate the spectra
sed = gal.stars.get_spectra(model)

# Get the observed spectra
gal.get_observed_spectra(cosmo)

# Set up the filter collection for imaging
filter_codes = [
    "JWST/NIRCam.F090W",
    "JWST/NIRCam.F150W",
    "JWST/NIRCam.F200W",
]
filters = Filters(filter_codes, new_lam=grid.lam)

gal.get_photo_lnu(filters)

AttributeError: 'numpy.ndarray' object has no attribute 'value' [EmissionModel.label: old_incident]

In [ ]:
# Define the width of the image
width = 30 * kpc

# Define image resolution (here we arbitrarily set it to 200
# pixels along an axis)
resolution = width / 200

print(f"Image width is {width:.2f} with {resolution:.2f} resolution")

In [ ]:
# Get the SPH kernel
sph_kernel = Kernel()
kernel_data = sph_kernel.get_kernel()

# Get the image
hist_imgs = gal.get_images_luminosity(
    resolution=resolution,
    fov=width,
    emission_model=model,
    img_type="hist",
    limit_to="nebular",  # we can limit to a single image type
)

# Get the image
smooth_imgs = gal.get_images_luminosity(
    resolution=resolution,
    fov=width,
    emission_model=model,
    img_type="smoothed",
    kernel=kernel_data,
    kernel_threshold=1,
)